## REFORMAT DATASET
Reformatting the layout of the overall dataset in terms of column information, tupled statsitical data, and identifying the null values.

In [12]:
import pandas as pd
import gzip

with gzip.open('./Storm Dataset/storm2008.csv.gz', 'rt', newline='') as csv_file:
    csv_data = csv_file.read()
    with open('storm2008.csv', 'wt') as out_file:
         out_file.write(csv_data)
    


In [ ]:
##Importing Libraries For Exploration
import pandas as pd

EnvImpact_Data = pd.read_csv("./CorporateEnvImpact_Dataset.csv")
EnvImpact_Data.head(5)